In [1]:
import math
import numpy as np
import re
from PIL import Image
import os
import struct
import matplotlib.pyplot as plt
import random
import time
import cv2 as cv

In [2]:
def load_mnist(path, kind="train"):
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
 
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        # 'I'表示一个无符号整数，大小为四个字节
        # '>II'表示读取两个无符号整数，即8个字节
        labels = np.fromfile(lbpath, dtype=np.uint8)
 
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols =  struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels

In [3]:
X_train, y_train = load_mnist("mnist_dataset/", kind="train")
X_test, y_test = load_mnist("mnist_dataset/", kind="t10k")

In [4]:
def get_act(x):
    act_vec = []
    for i in x:
        act_vec.append(1/(1+np.exp(-i)))
    act_vec = np.array(act_vec)
    return act_vec

In [5]:
class LoadNN(object):
    
    def __init__(self,filename):
        args = []
        with open(filename,'r') as f:
            while True:
                line = f.readline()
                if not line:
                    break
                pass
                args.append(line.strip())
        self.inp_num = int(args[0])
        self.hid_num = int(args[1])
        self.out_num = int(args[2])
        self.inp_lrate = float(args[3])
        self.hid_lrate = float(args[4])
        self.cost_time = []

        w_temp = []
        for i in range(5, 5 + self.inp_num):
            s1 = re.split(' ',args[i])
            for j in s1:
                w_temp.append(float(j))
        self.w1 = np.array(w_temp).reshape(self.inp_num, int(len(w_temp)/self.inp_num))
        print(self.w1.shape)

        w_temp = []
        s1 = re.split(' ', args[5 + self.inp_num])
        for j in s1:
            w_temp.append(float(j))
        self.w2 = np.array(w_temp).reshape(int(len(w_temp)/self.out_num),self.out_num)
        print(self.w2.shape)

        off_temp = []
        s1 = re.split(' ', args[5 + self.inp_num + 1])
        for j in s1:
            off_temp.append(float(j))
        self.hid_offset = np.array(off_temp)
        print(self.hid_offset.shape)

        off_temp = []
        s1 = re.split(' ', args[5 + self.inp_num + 2])
        for j in s1:
            off_temp.append(float(j))
        self.out_offset = np.array(off_temp)
        print(self.out_offset.shape)
    def getrs(self,img):
        img = np.array(img)
        hid_value = np.dot(img,self.w1) + self.hid_offset
        hid_act = get_act(hid_value)
        out_value = np.dot(hid_act,self.w2) + self.out_offset
        out_act = get_act(out_value)
        print('predict:',np.argmax(out_act))
    def error(self,X,y):
        X = np.array(X,dtype='float') 
        X/=256
        temp=0
        err_sample = []
        for count in range(len(X)):
            start_time = time.time()
            # 隐层值
            self.hid_value = np.dot(X[count], self.w1) + self.hid_offset 
            # 隐层激活值
            self.hid_act = get_act(self.hid_value)
            # 输出层值
            self.out_value = np.dot(self.hid_act, self.w2) + self.out_offset
            # 输出层激活值
            self.out_act = get_act(self.out_value)
            if np.argmax(self.out_act) == y[count]:
                temp+=1
            else:
                err_temp = []
                err_temp.append(y[count])
                err_sample.append(err_temp)
            end_time = time.time()
            self.cost_time.append(end_time - start_time)
            
        with open("err_label.txt",'w') as f:
            for item in err_sample:
                f.write(str(item))
                f.write('\n')
        print('Error is: %.2f%%'%((1-float(temp)/len(X))*100))

In [6]:
network = LoadNN("MyNetWork_mini_bath_128node_1.txt")

(784, 128)
(128, 10)
(128,)
(10,)


In [7]:
network.error(X_test,y_test)

Error is: 3.51%


In [8]:
image = cv.imread('demo_rev.jpg',cv.IMREAD_GRAYSCALE)
res = cv.resize(image,(28,28),interpolation=cv.INTER_CUBIC)

In [9]:
network.getrs(res.reshape(1,784))

predict: 3


In [11]:
X = np.array(res,dtype='float')
X = X.reshape(784,-1)
X = [0 if i < 16 else i/256 for i in X]
X = np.array(X,dtype='float')
X = X.reshape(1,784)
X = np.array(X,dtype='float')
network.getrs(X)

predict: 3
